In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
from IPython.display import FileLink

import base64
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

import urllib3
from ipywidgets import interact, fixed
import ipywidgets as widgets

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import pandas as pd
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
document = open('Document.txt').read()
list_of_sentence=document.split("\n")
list_of_simple_preprocess_data = []
for i in list_of_sentence:
    list_of_simple_preprocess_data.append(gensim.utils.simple_preprocess(i, deacc=True, min_len=2))
data = list_of_simple_preprocess_data
stopwords = open('stopwords.txt').read()
stopwords = set(stopwords.split())

In [5]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)


In [6]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stopwords] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [7]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [8]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


In [9]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { max-width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.output_area { max-width:100% !important; }</style>"))
display(HTML("<style>.input_area { max-width:100% !important; }</style>"))
style = {'description_width': 'initial'}

# Topic Model Visualization
Steps to follow -
1. Select the number of topics, minimum number of documents and document weight as required from dropdowns.
2. Visualization would be displayed including the Top 10 Most Sailent Terms
3. Choose the topic from the "Topic" dropdown to see the related Keywords, Weight and Document

More information about the visualization interface can be found <a href=" https://globalsers.org/visualization-interface-info/" target="_blank">here</a>

In [10]:
from ipywidgets import interact, fixed
import ipywidgets as widgets

a_list = list(range(2, 51))
Topics = widgets.Dropdown(
    options=a_list,
    value=15,
    style=style,
    description='Number of Topics:',
    disabled=False,
)

MinimumDocumentList = list(range(0, 11))
MinimumDocument = widgets.Dropdown(
    options=MinimumDocumentList,
    value=3,
    style=style,
    description='Select minimum number of documents:',
    disabled=False,
)

import decimal

def float_range(start, stop, step):
  while start < stop:
    yield float(start)
    start += decimal.Decimal(step)

float_range_list= (list(float_range(0, 1, '0.1')))

percent_dropdown = widgets.Dropdown(
    options=float_range_list,
    value=0.6,
    style=style,
    description='Select minimum document weight:',
    disabled=False,
)

@interact(Number=Topics,Percentage=percent_dropdown, Minimum_Document=MinimumDocument)
def my_function(Number, Percentage,Minimum_Document):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=Number, id2word=id2word,random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    pyLDAvis.enable_notebook()
    p = pyLDAvis.gensim.prepare(lda_model, corpus, id2word,R=10, sort_topics=False, mds='mmds')
    display(p)
    def format_topics_sentences(ldamodel=None, corpus=corpus, texts=list_of_sentence):
    # Init output
        sent_topics_df = pd.DataFrame()
        pd.set_option('max_colwidth',None)


        # Get main topic in each document
        for i, row_list in enumerate(ldamodel[corpus]):
            row = row_list[0] if ldamodel.per_word_topics else row_list            
            # print(row)
            row = sorted(row, key=lambda x: (x[1]), reverse=True)
            # Get the Dominant topic, Perc Contribution and Keywords for each document
            for j, (topic_num, prop_topic) in enumerate(row):
                if ((j == 0)):  # => dominant topic
                    wp = ldamodel.show_topic(topic_num)
                    topic_keywords = ", ".join([word for word, prop in wp])
                    sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
                else:
                    break
        sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

        # Add original text to the end of the output
        contents = pd.Series(texts)
        sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
        return(sent_topics_df)


    df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=list_of_sentence)

    # Format
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Weight', 'Keywords', 'Document']
    df_dominant_topic['Dominant_Topic'] = df_dominant_topic['Dominant_Topic'] + 1

    def edit_download_html(htmlWidget, filename, title = "Click here to download: "):

    # Change widget html temperarily to a font-awesome spinner
        htmlWidget.value = "<i class=\"fa fa-spinner fa-spin fa-2x fa-fw\"></i><span class=\"sr-only\">Loading...</span>"

        # Process raw data
        data = open(filename, "rb").read()
        b64 = base64.b64encode(data)
        payload = b64.decode()

        # Create and assign html to widget
        html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
        htmlWidget.value = html.format(payload = payload, title = title+filename, filename = filename)

    #ALL = 'ALL'
   
    def unique_sorted_values_plus_ALL(array):
        colvalue = array.unique().tolist()
        colvalue.sort()
        #colvalue.insert(0, ALL)
        return colvalue
    
    dominant_topics =(df_dominant_topic[df_dominant_topic.Weight > percent_dropdown.value])
    dominant_list =unique_sorted_values_plus_ALL(dominant_topics['Dominant_Topic'].value_counts().loc[lambda x : x>MinimumDocument.value].reset_index()['index'])
    if not dominant_list: 
        display('No dominant topic found')
    else:
        dominant_list = widgets.Dropdown(
        style=style,
        options = dominant_list,
        description='Topic:',
        disabled=False,
        )
        @interact(DominantTopic=dominant_list)
        def my_function(DominantTopic):
            df_dominant = (df_dominant_topic[df_dominant_topic['Dominant_Topic'] == DominantTopic])
            df_pecentage = df_dominant[df_dominant.Weight > percent_dropdown.value]
            df_topic = df_pecentage.sort_values(by=['Weight'],ascending=False)
            download = (df_topic[['Keywords', 'Weight','Document']])
    #       display(download)
            download['Keywords'] = download['Keywords'].mask(download['Keywords'].ne(download['Keywords'].shift()).cumsum().duplicated(), '')
            StyleDownload = download.style.hide_index()
            displayDF = StyleDownload.set_properties(**{'text-align': 'left'}).set_properties(subset=['Keywords'], **{'width': '300px'})
            display(displayDF.set_table_styles([dict(selector='th', props=[('text-align', 'left')])]))
            download.to_csv("output.csv", index=False)
            htmlWidget = widgets.HTML(value = '')
            display(htmlWidget)
            filename = r"output.csv"
            title = "Click here to download: "
            edit_download_html(htmlWidget, filename, title = title)
         

interactive(children=(Dropdown(description='Number of Topics:', index=13, options=(2, 3, 4, 5, 6, 7, 8, 9, 10,…

In [ ]:
import matplotlib.pyplot as plt
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        #model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=id2word,random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=50, step=1)
# Show graph
limit=50; start=2; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()
